# JuSpyce API test: group permutation test

In [1]:
import sys
import os
from glob import glob
import pathlib
import numpy as np
import pandas as pd
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt

# current path
wd = pathlib.Path().resolve().parent
print(wd)

# import juspyce
sys.path.append(os.path.dirname(os.path.join(wd, "juspyce")))
from juspyce.api import JuSpyce
from juspyce.stats import *
from juspyce.utils import *

/Users/llotter/projects/juspyce


## Load JuSpyce data from test_juspyce.fit.ipynb

In [2]:
juspyce_vol = JuSpyce.from_pickle(os.path.join(wd, "testing", "test_juspyce_vol.pkl.gz"))

INFO:juspyce.api:Loaded complete object from /Users/llotter/projects/juspyce/testing/test_juspyce_vol.pkl.gz.


## Permutation of group assignment to compare predictions between groups

This is based on the idea of the original [JuSpace](https://github.com/juryxy/JuSpace) toolbox. If group differences in a certain imaging modality have biological meaning, they may align with the distribution of a certain neurotransmitter (e.g., see in the JuSpace paper: the difference in rsfMRI activity between patients with Parkinson's and healthy controls aligns with dopaminergic transmitter maps).
Two groups are defined via a vector with the length of the "Y" dataframe. The difference between two groups (mean difference, Cohen's d, every vector in group A - mean of group B, ...) is calculated (`JuSpyce.transform()`) and a prediction function (`JuSpyce.predict()`) is applied. The group labels are permuted `n_perm` times and the transform and prediction process is repeated to generate null distributions of "prediction values" ($R^2$, correlation coefficients, ...). From these, p values are calculated.

### Grouping variable

In [3]:
n_Y = juspyce_vol.Y.shape[0]
groups = [0] * int(n_Y/2) + [1] * int(n_Y/2)
print(n_Y, groups)

28 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Does the mean difference between two groups correlate with a certain transmitter?

mean difference of two groups -> one vector 

In [4]:
# permute groups
p_data, true_predictions, null_predictions = juspyce_vol.permute_groups(
    method="spearman", 
    comparison="diff(mean(A),mean(B))", 
    groups=groups,
    n_perm=1000, 
    p_tail="two",
    r_to_z=True, adjust_r2=True, mlr_individual=True,
    n_proc=8, n_proc_predict=1, seed=None,
    verbose=True, store=True)

INFO:juspyce.api:Running 'true' group comparison and prediction (comparison = 'diff(mean(A),mean(B))', method = 'spearman').
INFO:juspyce.api:Subtracting parcelwise mean of B from mean of A: new Y = mean(Y[A]) - mean(Y[B]).


Predicting (spearman, 1 proc):   0%|          | 0/1 [00:00<?, ?it/s]

INFO:juspyce.api:Running null comparisons and predictions (comparison = 'diff(mean(A),mean(B))', method = 'spearman').


Null comparisons (spearman, 8 proc):   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:juspyce.api:Calculating exact p-values (tails = 'two').


In [5]:
# look at the result
juspyce_vol.p_comparisons.keys()

dict_keys(['diff(mean(A),mean(B))-spearman'])

In [6]:
juspyce_vol.p_comparisons["diff(mean(A),mean(B))-spearman"]

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"diff(mean(A),mean(B))",0.25,0.688,0.772,0.53,0.238,0.694,0.76


### Can the effect size between two groups be predicted from a certain transmitter?

In [7]:
# compare
juspyce_vol.compare(
    comparison="cohen(A,B)",
    groups=groups
)
# predict
juspyce_vol.predict(
    method="dominance",
    comparison="cohen(A,B)"
)
# permute groups
p_data, true_predictions, null_predictions = juspyce_vol.permute_groups(
    method="dominance", 
    comparison="cohen(A,B)", 
    groups=groups,
    n_perm=1000, 
    p_tail="two",
    r_to_z=True, adjust_r2=True, mlr_individual=True,
    n_proc=8, n_proc_predict=1, seed=None,
    verbose=True, store=True)

INFO:juspyce.api:Calculating parcelwise effect size between A and B (cohen, paired: False).


  0%|          | 0/116 [00:00<?, ?it/s]

Predicting (dominance, 8 proc):   0%|          | 0/1 [00:00<?, ?it/s]

INFO:juspyce.api:Running 'true' group comparison and prediction (comparison = 'cohen(A,B)', method = 'dominance').
INFO:juspyce.api:Calculating parcelwise effect size between A and B (cohen, paired: False).


  0%|          | 0/116 [00:00<?, ?it/s]

Predicting (dominance, 1 proc):   0%|          | 0/1 [00:00<?, ?it/s]

INFO:juspyce.api:Running null comparisons and predictions (comparison = 'cohen(A,B)', method = 'dominance').


Null comparisons (dominance, 8 proc):   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:juspyce.api:Calculating exact p-values (tails = 'two').


In [8]:
# look at the result
juspyce_vol.p_comparisons.keys()

dict_keys(['diff(mean(A),mean(B))-spearman', 'cohen(A,B)-dominance_total', 'cohen(A,B)-dominance_individual', 'cohen(A,B)-dominance_relative', 'cohen(A,B)-dominance_full_r2'])

In [9]:
display(juspyce_vol.comparisons["cohen(A,B)"])
display(juspyce_vol.predictions["cohen(A,B)-dominance_total"])
display(juspyce_vol.p_comparisons["cohen(A,B)-dominance_total"])

,LH_Vis_1,LH_Vis_2,LH_Vis_3,LH_Vis_4,LH_Vis_5,LH_Vis_6,LH_Vis_7,LH_Vis_8,LH_Vis_9,LH_SomMot_1,...,PUT-rh,CAU-rh,HIP-lh,AMY-lh,pTHA-lh,aTHA-lh,NAc-lh,GP-lh,PUT-lh,CAU-lh
"cohen(A,B)",0.276661,-0.517828,-0.696612,-0.325058,-0.60921,-0.590054,-0.575566,-0.433675,-0.551265,0.381074,...,0.231055,0.146563,0.221554,-0.316784,0.543487,0.423675,0.000933,0.213862,0.331783,0.157934


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"cohen(A,B)",0.038337,0.000977,0.023584,0.021675,0.082861,0.005747,0.018984


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"cohen(A,B)",0.808,0.232,0.944,0.722,0.14,0.988,0.732


### Can the individual differencec between the individuals of one group and the mean of another ("reference") be predicted from a certain transmitter?

In [12]:
# compare
juspyce_vol.compare(
    comparison="diff(A,mean(B))",
    groups=groups
)
# predict
juspyce_vol.predict(
    method="slr",
    comparison="diff(A,mean(B))"
)
# permute groups
p_data, true_predictions, null_predictions = juspyce_vol.permute_groups(
    method="slr", 
    comparison="diff(A,mean(B))", 
    groups=groups,
    n_perm=1000, 
    p_tail="two",
    r_to_z=True, adjust_r2=True, mlr_individual=True,
    n_proc=8, n_proc_predict=1, seed=None,
    verbose=True, store=True)

INFO:juspyce.api:Subtracting parcelwise mean of B from A: new Y = Y[A] - mean(Y[B]).


Predicting (slr, 8 proc):   0%|          | 0/14 [00:00<?, ?it/s]

INFO:juspyce.api:Running 'true' group comparison and prediction (comparison = 'diff(A,mean(B))', method = 'slr').
INFO:juspyce.api:Subtracting parcelwise mean of B from A: new Y = Y[A] - mean(Y[B]).


Predicting (slr, 1 proc):   0%|          | 0/14 [00:00<?, ?it/s]

INFO:juspyce.api:Running null comparisons and predictions (comparison = 'diff(A,mean(B))', method = 'slr').


Null comparisons (slr, 8 proc):   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:juspyce.api:Calculating exact p-values (tails = 'two').


In [13]:
display(juspyce_vol.comparisons["diff(A,mean(B))"])
display(juspyce_vol.predictions["diff(A,mean(B))-slr"])
display(juspyce_vol.p_comparisons["diff(A,mean(B))-slr"])

,LH_Vis_1,LH_Vis_2,LH_Vis_3,LH_Vis_4,LH_Vis_5,LH_Vis_6,LH_Vis_7,LH_Vis_8,LH_Vis_9,LH_SomMot_1,...,PUT-rh,CAU-rh,HIP-lh,AMY-lh,pTHA-lh,aTHA-lh,NAc-lh,GP-lh,PUT-lh,CAU-lh
control,-0.696382,-0.729955,-0.718457,-0.956186,-1.342141,-0.859032,-1.318171,-0.405459,-0.457808,-0.864569,...,0.249007,0.003555,-0.663448,-1.002723,1.549940,0.513183,-1.355185,0.408726,0.815779,-0.519336
touch,-0.326454,-0.968939,-0.435304,-0.913205,-1.033309,-0.625035,-0.906001,-0.832693,-0.687413,1.393019,...,-0.359452,-0.552504,-0.437790,-0.700949,0.275985,-0.126077,-0.708779,-1.077243,-0.675590,-0.543704
interoception,-0.300974,-0.655026,-0.435491,-0.775447,-0.990875,-0.217356,-0.904939,-1.016064,-0.735211,1.071240,...,2.626827,0.351506,-0.129209,0.563854,-0.055587,-0.031093,-0.027151,-0.054901,0.474340,0.083803
learning,2.165139,-0.062543,0.055625,0.067692,-0.410888,0.022266,-0.527621,-0.096610,-0.231291,-1.041584,...,1.397316,1.678663,3.115950,0.239604,0.208409,0.796856,3.318753,2.848715,2.562147,2.141754
attention,-0.759810,0.642890,-0.007354,1.177492,0.852357,0.249951,0.796263,1.379926,0.519048,-0.587355,...,-0.862062,-0.676476,-1.126829,-1.850535,-0.634298,-0.558722,-1.563317,-1.339263,-1.241443,-0.770026
language,-0.078092,-0.564090,-0.590121,-0.092469,-1.260254,-0.399269,0.134711,-0.870407,-0.612218,0.778460,...,-0.537063,0.013762,-0.481074,-0.905942,-0.701649,0.153871,-0.131010,-0.081346,-0.265892,0.176372
interaction,0.845897,0.505428,-0.013189,0.924966,0.649225,-0.118451,0.124711,0.300235,-0.257690,2.323135,...,-0.123388,-0.383490,1.198117,-0.800753,0.330357,-0.515459,-1.055859,0.026500,0.172668,-0.904517
inhibition,-0.945837,-1.147676,-0.681386,-0.921550,-1.422752,-0.966913,-1.228442,-0.827222,-0.735634,-0.144012,...,0.193167,0.109954,-1.078434,-1.276222,0.587552,-0.154345,-1.097669,0.410472,0.797011,-0.430287
somatosensory,-0.337952,-1.037122,-0.448577,-0.923679,-1.083739,-0.587661,-0.971116,-0.903434,-0.724640,1.434588,...,-0.386976,-0.697618,-0.460913,-0.738152,0.014388,-0.285712,-0.647586,-1.189883,-0.687300,-0.542263
decision,-0.641819,-0.147905,-0.615453,-0.034513,-0.924296,-0.774381,0.503671,-0.350625,-0.556892,-1.363142,...,0.425344,1.849352,-1.050695,-1.030874,-0.440526,0.840736,4.074118,2.923260,2.049824,2.361438


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.126291,0.009378,0.024570,-0.008127,0.140611,-0.007377,-0.004387
touch,0.000855,0.033290,0.037054,-0.007587,-0.008158,0.007133,0.032149
interoception,-0.008714,0.102674,0.135328,0.130236,-0.002074,-0.002777,0.039475
learning,0.262379,0.174698,0.132224,0.115110,0.237950,-0.006658,0.466333
attention,0.075697,0.185698,-0.008709,0.254873,0.036221,-0.004796,0.135009
language,0.110190,0.001940,0.040966,-0.001053,0.011124,0.006515,-0.001887
interaction,-0.007874,-0.007333,0.021013,0.132108,-0.006975,-0.004007,0.007021
inhibition,0.080298,0.019310,-0.008442,-0.008768,0.071378,0.002215,-0.006424
somatosensory,-0.004691,0.040123,0.055824,-0.007602,-0.008705,0.002762,0.036245
decision,0.046324,0.026507,-0.002498,0.246366,0.141452,0.151389,0.366907


,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.472,0.284,0.526,0.204,0.346,0.558,0.312
touch,0.894,0.386,0.662,0.146,0.146,0.442,0.642
interoception,0.030,0.880,0.326,0.806,0.174,0.950,0.910
learning,0.042,0.298,0.114,0.846,0.018,0.374,0.062
attention,0.714,0.342,0.048,0.332,0.816,0.524,0.426
language,0.450,0.522,0.986,0.494,0.502,0.832,0.276
interaction,0.174,0.194,0.754,0.804,0.134,0.928,0.578
inhibition,0.798,0.620,0.054,0.008,0.814,0.722,0.220
somatosensory,0.456,0.700,0.990,0.024,0.034,0.748,0.882
decision,0.868,0.576,0.652,0.698,0.056,0.001,0.172


## Correct p-values

p values can be corrected across dataframes or rows/columns of dataframes using `JuSpyce.correct_p()`. The method will, if not provided differently, loop over all p-value dataframes and apply multiple comparison correction methods from `statsmodels.stats.multitest.multipletests`.

Results will be stored in the `JuSpyce.p_comparisons` dict as `JuSpyce.p_comparisons["comparison_name-prediction_name--correction_method"]`, e.g., if comparison is `cohen(A,B)`, method is `spearman` and correction is `fdr_bh`: `juspyce_vol.p_predictions["cohen(A,B)-spearman--fdr_bh"]`

In [14]:
juspyce_vol.correct_p(
    analysis="comparisons", # one of "predictions" or "comparisons" -> here: predictions
    method="all", # if all, iterate over all dataframes (but calculate values for each individual dataframe)
    mc_alpha=0.05, # alpha treshold, should have no effect
    mc_method="fdr_bh", # correction method passed to statsmodels
    mc_dimension="array") # 'array', 'row' or 'column'
for k in juspyce_vol.p_comparisons:
    display(k)
    display(juspyce_vol.p_comparisons[k])

'diff(mean(A),mean(B))-spearman'

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"diff(mean(A),mean(B))",0.25,0.688,0.772,0.53,0.238,0.694,0.76


'cohen(A,B)-dominance_total'

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"cohen(A,B)",0.808,0.232,0.944,0.722,0.14,0.988,0.732


'cohen(A,B)-dominance_individual'

...

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
"cohen(A,B)",0.9296,0.9296,0.084,0.992,0.9296,0.9296,0.949667


'diff(A,mean(B))-slr'

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.472,0.284,0.526,0.204,0.346,0.558,0.312
touch,0.894,0.386,0.662,0.146,0.146,0.442,0.642
interoception,0.030,0.880,0.326,0.806,0.174,0.950,0.910
learning,0.042,0.298,0.114,0.846,0.018,0.374,0.062
attention,0.714,0.342,0.048,0.332,0.816,0.524,0.426
language,0.450,0.522,0.986,0.494,0.502,0.832,0.276
interaction,0.174,0.194,0.754,0.804,0.134,0.928,0.578
inhibition,0.798,0.620,0.054,0.008,0.814,0.722,0.220
somatosensory,0.456,0.700,0.990,0.024,0.034,0.748,0.882
decision,0.868,0.576,0.652,0.698,0.056,0.001,0.172


'diff(A,mean(B))-slr--fdr_bh'

,5HT2a-cimbi36-29-beliveau2017,NMDA-ge179-29-galovic2021,mGluR5-abp688-73-smart2019,MU-carfentanil-204-kantonen2020,GABAa-flumazenil-6-dukart2018,5HT1b-p943-65-gallezot2010,D2-raclopride-156-malen2022
control,0.797517,0.679163,0.818222,0.540324,0.686000,0.854438,0.686000
touch,0.942065,0.727462,0.926800,0.480516,0.480516,0.784000,0.925235
interoception,0.238000,0.942065,0.686000,0.942065,0.501529,0.969792,0.948723
learning,0.257250,0.679163,0.446880,0.942065,0.176400,0.718667,0.276182
attention,0.942065,0.686000,0.261333,0.686000,0.942065,0.818222,0.773111
language,0.784000,0.818222,0.990000,0.818222,0.818222,0.942065,0.679163
interaction,0.501529,0.528111,0.942065,0.942065,0.480516,0.957305,0.858242
inhibition,0.942065,0.906866,0.261333,0.130667,0.942065,0.942065,0.567368
somatosensory,0.784000,0.942065,0.990000,0.213818,0.238000,0.942065,0.942065
decision,0.942065,0.858242,0.926029,0.942065,0.261333,0.032667,0.501529
